
# 🌍 Analyzing Hourly GPM IMERG Precipitation L3


In [ ]:
import earthaccess 

print(f"using earthaccess version {earthaccess.__version__}")

auth = earthaccess.login(strategy="netrc")
# are we authenticated?
if not auth.authenticated:
    # ask for credentials and persist them in a .netrc file
    auth.login(strategy="interactive", persist=True)

### Authentication in the cloud 
If the collection is a cloud-hosted collection we can print the `summary()` and get the S3 credential endpoint. These S3 credentials are temporary and we can use them with third party libraries such as s3fs, boto3 or awscli.

In [ ]:
from pprint import pprint
results = earthaccess.search_datasets(keyword = "IMERGHH Precipitation L3 ")

# Let's print our collections
for collection in results:
    # pprint(collection.summary())
    print(pprint(collection.summary()) ,"\n")

## A year of data 

Things to keep in mind:

* this is a high resolution dataset
* this is a global dataset
* temporal resolution is 30 minutes = 48 granules per day

With earthaccess we can programatically download this data (or access it directly in the cloud) but for the size of the dataset we have to batch our requests or reduce the scope for this example.

In [ ]:
# count = 1 is important so we don't download the whole year! 
granules = earthaccess.search_data(count=1,
                                   concept_id="C1598621093-GES_DISC",
                                   temporal = ("2020-01","2021-01"))

## Working with the URLs directly

If we chose, we can use `earthdata` to grab the file's URLs and then download them with another library (if we have a `.netrc` file configured with NASA's EDL credentials)
Getting the links to our data is quiet simple with the `data_links()` method on each of the results:

In [ ]:
# on_prem is a bit misleading, the collection is cloud hosted, in this case the access can be done out of region 
granules[0].data_links(access="out-of-region")

In [ ]:
granules[0].data_links(access="direct")

## Downloading data

We are going to download 1 granule.


In [ ]:
local_path = "./data"
files = earthaccess.download(granules[0:1], local_path=local_path)
files

In [ ]:
from jupyterlab_h5web import H5Web
from pathlib import Path

file = Path(local_path) / Path(files[0])
H5Web(file)

In [ ]:
%%time

results = []

# we just grab 1 granule from May for each year of the dataset
for year in range(2018, 2022):
    granules = earthaccess.search_data(
        concept_id="C1598621093-GES_DISC",
        temporal=(f"{year}-05-01", f"{year}-05-02"))
    if len(granules)>0:
        results.extend(granules)

print(len(results))

In [ ]:
results[0]

In [ ]:
file = earthaccess.open(results[0:1])

## Plotting 

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

with h5py.File(file[0], 'r') as h5file:
    img_arr = np.transpose(h5file['/Grid/HQprecipitation'][0,:,:])
    
fig = plt.subplots(figsize=(10,10)) 
plt.title('Plot raw 3IMERG image data')
im = plt.imshow(img_arr) 
plt.colorbar(im)